In [1]:
import os
import gc
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import matplotlib.pyplot as plt

np.random.seed(0)
gc.enable()

D:\Anaconda\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
train = pd.read_csv("../data/train_small.csv")
train = train[train['maxPlace'] > 1]

In [3]:
target = "winPlacePerc"
features = list(train.columns)
features.remove("Id")
features.remove("matchId")
features.remove("groupId")
features.remove("matchType")

y = np.array(train[target])
features.remove(target)
x = train[features]

del train
gc.collect()

14

In [4]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=.1, random_state=0)

del x, y
gc.collect()

0

In [5]:
ran_forest = RandomForestRegressor(n_estimators=50, n_jobs=4, random_state=0)

In [6]:
%%time
ran_forest.fit(x_train, y_train)

Wall time: 58.8 s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=4,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [7]:
print('Training MAE: ', metrics.mean_absolute_error(ran_forest.predict(x_train), y_train))
print('Validation MAE: ', metrics.mean_absolute_error(ran_forest.predict(x_val), y_val))

Training MAE:  0.02285125709424394
Validation MAE:  0.060770750309022216


In [8]:
feature_importance = pd.DataFrame(ran_forest.feature_importances_, index=x_train.columns, columns=['importance']).sort_values('importance', ascending=False)
del ran_forest
gc.collect()

24

In [9]:
x_values = list(range(len(feature_importance.index)))
fig = plt.figure(figsize=(8,6))
plt.style.use('fivethirtyeight')
plt.bar(x_values, feature_importance['importance'])
plt.xticks(x_values, np.array(feature_importance.index), rotation='vertical')
plt.ylabel('Importance')
plt.xlabel('Feature')
plt.title('Feature Importances')
plt.tight_layout()
plt.savefig('viz/small_RF_importances.png', dpi=fig.dpi)
plt.close()